In [1]:
from prepared_data.get_prepared_data import  get_prepared_data
import pandas as pd


In [2]:
train_org = pd.read_csv('../data/train.csv')
train, league_mean_ratting_all, teams_ratting_all  = get_prepared_data(train_org, number_of_history_matches= 8,)
train

/home/andrzej/PycharmProjects/Physionet_Challenge_2021/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


correction col type..
percent of object with nan value:  42.20
remove coach columns..
percent of object with nan value:  18.64
remove object with more that  col 80 empty...
percent of object with nan value:  18.64
remove history columns
percent of object with nan value:  15.11
remove all nan data
percent of object with nan value:  15.11


,target,is_cup,home_team_history_match_date_1,home_team_history_match_date_2,home_team_history_match_date_3,home_team_history_match_date_4,home_team_history_match_date_5,home_team_history_match_date_6,home_team_history_match_date_7,home_team_history_match_date_8,...,home_team_history_target_7_1.0,away_team_history_target_7_-1.0,away_team_history_target_7_0.0,away_team_history_target_7_1.0,home_team_history_target_8_-1.0,home_team_history_target_8_0.0,home_team_history_target_8_1.0,away_team_history_target_8_-1.0,away_team_history_target_8_0.0,away_team_history_target_8_1.0
id,,,,,,,,,,,,,,,,,,,,,
11989265,-1,1,78,258,75,193,67,103,88,74,...,1,0,0,1,1,0,0,0,1,0
16865249,1,1,219,168,286,71,49,3145,67,76,...,0,0,0,1,0,0,1,0,0,1
16916583,1,1,94,627,93,99,93,74,168,69,...,0,1,0,0,0,1,0,0,0,1
16940078,-1,1,167,168,119,123,117,121,96,96,...,1,0,0,1,0,0,1,0,0,1
17213503,1,1,100,92,76,144,92,126,238,72,...,0,0,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17882059,1,1,504,1135,712,168,168,168,505,168,...,0,0,1,0,0,1,0,1,0,0
17887522,-1,1,127,167,208,72,96,127,545,1133,...,0,1,0,0,1,0,0,1,0,0
17882061,-1,1,545,1133,715,168,168,168,505,168,...,0,0,0,1,1,0,0,0,0,1


In [3]:
import  re
used = set()
columns_only_first_hist = [column for column in train.columns.values if
                           re.sub('\d', '', column) not in used and (used.add(re.sub('\d', '', column)) or True)]
print(columns_only_first_hist)

['target', 'is_cup', 'home_team_history_match_date_1', 'home_team_history_is_play_home_1', 'home_team_history_is_cup_1', 'home_team_history_goal_1', 'home_team_history_opponent_goal_1', 'home_team_history_rating_1', 'home_team_history_opponent_rating_1', 'away_team_history_match_date_1', 'away_team_history_is_play_home_1', 'away_team_history_is_cup_1', 'away_team_history_goal_1', 'away_team_history_opponent_goal_1', 'away_team_history_rating_1', 'away_team_history_opponent_rating_1', 'home_team_history_gol_difference_1', 'home_team_mean_regeneration_time', 'away_team_history_gol_difference_1', 'away_team_mean_regeneration_time', 'home_team_mean_hist_rat', 'home_team_mean_all_ratting', 'away_team_mean_hist_rat', 'away_team_mean_all_ratting', 'league_id_ratting', 'sum_history_targets', 'home_team_history_target_1_-1.0', 'home_team_history_target_1_0.0', 'away_team_history_target_1_-1.0', 'away_team_history_target_1_0.0']


In [4]:
test_org = pd.read_csv('../data/test.csv')
test,_,_  = get_prepared_data(test_org, 8, league_mean_ratting_all, teams_ratting_all )
test.isnull().values.any()

correction col type..
percent of object with nan value:  39.35
remove coach columns..
percent of object with nan value:  13.93
remove object with more that  col 80 empty...
percent of object with nan value:  13.93
remove history columns
percent of object with nan value:  11.15
remove all nan data
percent of object with nan value:  11.15


False

# Dodanie cech
- Sredni całkowity ranking drużyny
- Sredni ranking drużyny ostatni n meczów
- Sredni ranking ligi
- Intensywnosc druzyny, na podstawie dat rozegranych meczów
- Historyczny mecz target, na postawie bramek
- Róznica bramek
- Suma historycznych targetow  (




# Dobór ilości dni wstecz

In [5]:
from prepared_data.completed_missing_data import remove_history_col
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression

for i in range(9):
    data = remove_history_col(train,i)
    print(f'number of history columns {i}')

    for scaler in [StandardScaler, MinMaxScaler]:
        pipe = make_pipeline(scaler(), LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr',max_iter=1000))

        # pipe.fit(X_train, y_train)
        # results = pipe.predict(X_train)
        scores = cross_val_score(pipe,data.iloc[:, 1:], data.iloc[:, 0], cv=5, n_jobs=-1)
        print(pipe.named_steps)
        print("%0.4f accuracy with a standard deviation of %0.4f" % (scores.mean(), scores.std()))
    # print(f'accuracy: {pipe.score(X_test, y_test):.04f}')
# drzewach, svm
# kolumny
# kaggle
#

number of history columns 0
{'standardscaler': StandardScaler(), 'logisticregression': LogisticRegression(max_iter=1000, multi_class='ovr', random_state=0)}
0.4962 accuracy with a standard deviation of 0.0126
{'minmaxscaler': MinMaxScaler(), 'logisticregression': LogisticRegression(max_iter=1000, multi_class='ovr', random_state=0)}
0.4962 accuracy with a standard deviation of 0.0126
number of history columns 1
{'standardscaler': StandardScaler(), 'logisticregression': LogisticRegression(max_iter=1000, multi_class='ovr', random_state=0)}
0.4974 accuracy with a standard deviation of 0.0115
{'minmaxscaler': MinMaxScaler(), 'logisticregression': LogisticRegression(max_iter=1000, multi_class='ovr', random_state=0)}
0.4972 accuracy with a standard deviation of 0.0117
number of history columns 2
{'standardscaler': StandardScaler(), 'logisticregression': LogisticRegression(max_iter=1000, multi_class='ovr', random_state=0)}
0.4979 accuracy with a standard deviation of 0.0108
{'minmaxscaler': Mi

# Dobór cech

In [10]:
from sklearn.feature_selection import SequentialFeatureSelector

data = remove_history_col(train,6)

svc = make_pipeline(StandardScaler(), LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr', max_iter=1000))

sfs = SequentialFeatureSelector(svc, n_features_to_select=5,cv= 5, n_jobs=-1)
sfs.fit(data.iloc[:, 1:], data.iloc[:, 0])

data.columns[ sfs.get_support()]

IndexError: boolean index did not match indexed array along dimension 0; dimension is 105 but corresponding boolean dimension is 104

In [18]:
print(data.iloc[:, 1:].columns[sfs.support_])

Index(['home_team_mean_all_ratting', 'away_team_mean_all_ratting',
       'home_team_history_opponent_rating_1',
       'away_team_history_opponent_rating_5',
       'home_team_history_gol_difference_3'],
      dtype='object')


In [20]:
from sklearn.feature_selection import RFECV
data = remove_history_col(train,6)
selector = RFECV( LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr', max_iter=1000), step=5, cv=5)
selector = selector.fit(data.iloc[:, 1:], data.iloc[:, 0])
data.columns[ selector.get_support()]

/home/andrzej/PycharmProjects/Physionet_Challenge_2021/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/andrzej/PycharmProjects/Physionet_Challenge_2021/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
 

IndexError: boolean index did not match indexed array along dimension 0; dimension is 105 but corresponding boolean dimension is 104